# Create Feature set

In [2]:
from tqdm import tqdm
import numpy as np
import csv
import os
from PIL import Image
import matplotlib.pyplot as plt
from glob import glob
from keras.models import Model
from keras.layers import Input, merge, Conv2D, MaxPooling2D, UpSampling2D, GlobalMaxPooling2D
from keras.optimizers import Adam
from keras.optimizers import SGD
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, EarlyStopping, ReduceLROnPlateau
from keras import backend as K
from random import uniform
import bcolz
from time import *
from keras.layers import merge
from keras.layers.core import Lambda
from keras.models import Model
from keras.models import Sequential
from keras.layers.normalization import BatchNormalization
from keras.layers.core import Dropout
from keras.layers import Input, Dense
import math
import pandas as pd
from keras.utils import np_utils as u
import utils; reload(utils)
from utils import *
import resnet50; reload(resnet50)
from resnet50 import Resnet50
from keras.preprocessing.image import ImageDataGenerator
import myFunctionsv3; reload(myFunctionsv3)
from myFunctionsv3 import *
from  keras.applications.resnet50 import ResNet50
from squeezenet import SqueezeNet
#from densenet import DenseNet, DenseNetImageNet121

import gc

%matplotlib inline

# SqueezeNet

Requirement keras 2.0

Create a base model using squeezenet and initialize with imagenet weights. Add a lambda layer at the beginning to normalize and remove the fully connected layers at the end

In [25]:
imagenet_mean = np.array([123.68, 116.779, 103.939], dtype=np.float32).reshape(1,1,3)

def squeezenet_preprocess(x):
    """
        Subtracts the mean RGB value, and transposes RGB to BGR.
        The mean RGB was computed on the image set used to train the RESNET model.

        Args: 
            x: Image array (height x width x channels)
        Returns:
            Image array (height x width x transposed_channels)
    """
    x = x - imagenet_mean
    #x = x[...,[2,1,0]] # reverse axis rgb->bgr
    x = x[..., ::-1]

    return x  

In [26]:
model = Sequential()
model.add(Lambda(squeezenet_preprocess, input_shape=(360,640,3), output_shape=(360,640,3)))

In [27]:
squeezenet = SqueezeNet(input_shape=(360,640,3))

In [28]:
model.add(Model(inputs=squeezenet.input, outputs=squeezenet.layers[-6].output))
model.compile(SGD(0.05), 'binary_crossentropy', metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lambda_11 (Lambda)           (None, 360, 640, 3)       0         
_________________________________________________________________
model_2 (Model)              (None, 21, 39, 512)       722496    
Total params: 722,496.0
Trainable params: 722,496.0
Non-trainable params: 0.0
_________________________________________________________________


Precompute the features and save to disk to save training time

In [29]:
def createFeatures(path,name):
    rgb_hires = load_array(path + name + '_hires.dat')
    conv_feat = model.predict(rgb_hires, batch_size=64, verbose=0)
    save_array('/home/ubuntu/features/'+name+'_sqznt.dat',conv_feat)
    del(conv_feat,rgb_hires)
    gc.collect()

In [ ]:
files = ['trainfold0','trainfold1','trainfold3','trainfold4', 'trainfold5'] 
for i in tqdm(range(len(files))):
    createFeatures('/cat/home/ubuntu/cat/in/train/folds/',files[i])

  0%|          | 0/9 [00:00<?, ?it/s]

In [ ]:
files = ['train03_val_minus_0p5','train04_val_minus_0p5','train10_val_minus_0p5','train18_val_minus_0p5',
         'train21_val_minus_0p5','train14_val_minus_0p5','train19A_val_minus_0p5','train19B_val_minus_0p5','train24_val_minus_0p5',
        'train03top_left_val_minus_0p5','train04top_left_val_minus_0p5','train10top_left_val_minus_0p5','train18top_left_val_minus_0p5',
         'train21top_left_val_minus_0p5','train14top_left_val_minus_0p5','train19Atop_left_val_minus_0p5','train19Btop_left_val_minus_0p5','train24top_left_val_minus_0p5',
        'train03top_right_val_minus_0p5','train04top_right_val_minus_0p5','train10top_right_val_minus_0p5','train18top_right_val_minus_0p5',
         'train21top_right_val_minus_0p5','train14top_right_val_minus_0p5','train19Atop_right_val_minus_0p5','train19Btop_right_val_minus_0p5','train24top_right_val_minus_0p5',
        'train03bottom_left_val_minus_0p5','train04bottom_left_val_minus_0p5','train10bottom_left_val_minus_0p5','train18bottom_left_val_minus_0p5',
         'train21bottom_left_val_minus_0p5','train14bottom_left_val_minus_0p5','train19Abottom_left_val_minus_0p5','train19Bbottom_left_val_minus_0p5','train24bottom_left_val_minus_0p5',
        'train03bottom_right_val_minus_0p5','train04bottom_right_val_minus_0p5','train10bottom_right_val_minus_0p5','train18bottom_right_val_minus_0p5',
         'train21bottom_right_val_minus_0p5','train14bottom_right_val_minus_0p5','train19Abottom_right_val_minus_0p5','train19Bbottom_right_val_minus_0p5','train24bottom_right_val_minus_0p5'
        ]

for i in tqdm(range(len(files))):
    createFeatures('/cat/home/ubuntu/cat/in/train/hires/',files[i])

# DenseNet

Requirement keras 2.0.8

In [ ]:
dense = DenseNetImageNet121(input_shape=(360,640,3),weights='imagenet',include_top=False)

In [ ]:
model = Model(inputs = dense.input, outputs = dense.layers[-18].output)
model.summary()

In [ ]:
def createDenseNetFeatures(path,name):
    rgb_hires = load_array(path + name + '_hires.dat')
    conv_feat = model.predict(rgb_hires, batch_size=32, verbose=0)
    save_array('/cat/home/ubuntu/cat/out/features/train/'+name+'_dnsnt.dat',conv_feat)
    del(conv_feat,rgb_hires)
    gc.collect()

In [ ]:
files = ['trainfold1','trainfold2','trainfold3','trainfold4','trainfold5',
         'trainfold6','trainfold7','trainfold8','trainfold9','trainfold0'] 

for i in tqdm(range(len(files))):
    createDenseNetFeatures('/cat/home/ubuntu/cat/in/train/folds/',files[i])

In [ ]:
files = ['train03_val_minus_0p5','train04_val_minus_0p5','train10_val_minus_0p5','train18_val_minus_0p5',
         'train21_val_minus_0p5','train14_val_minus_0p5','train19_val_minus_0p5','train24_val_minus_0p5']
for i in tqdm(range(len(files))):
    createDenseNetFeatures('/cat/home/ubuntu/cat/in/train/hires/',files[i])